Cоздать модель, которая предсказывает, какие пассажиры выжили при кораблекрушении "Титаника".

Набор данных train_titanic.csv содержит подробную информацию о подмножестве пассажиров на борту и покажет, выжили они или нет.

Набор данных test_titanic.csv содержит аналогичную информацию, но не раскрывает «основную истину» для каждого пассажира. Задача - предсказать эти результаты.


In [1]:
import numpy as np
import pandas as pd

In [2]:
#  чтение данных
path_train_titanic = "/content/drive/MyDrive/Colab Notebooks/data/train_titanic.csv"
path_test_titanic = "/content/drive/MyDrive/Colab Notebooks/data/test_titanic.csv"
train_data = pd.read_csv(path_train_titanic)
test_data = pd.read_csv(path_test_titanic)

In [3]:
X_full=train_data
X_test_full=test_data

In [4]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
missing_values_count = train_data.isnull().sum()
print(missing_values_count)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [6]:
# отделяем столбец target
y = X_full['Survived']

In [7]:
# удаляем столбец target из тренировочных данных
drop_cols_X_full=['Survived']
X_full.drop(drop_cols_X_full,axis=1, inplace=True)

In [8]:
# разделим набор данных на обучааааащий и проверочный
from sklearn.model_selection import train_test_split
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, train_size = 0.8, test_size = 0.2, random_state = 0)

In [9]:
# выберем категорийные столбцы с малой мощьностью
cat_cols=[cname for cname in X_train_full.columns 
          if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == 'object']

In [10]:
cat_cols

['Sex', 'Embarked']

In [11]:
# выберем числовые столбцы
num_cols=[cname for cname in X_train_full.columns 
          if X_train_full[cname].dtype in ['int64', 'float64']]

In [12]:
num_cols

['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [13]:
# оставить только выбранные столбцы
my_cols = cat_cols + num_cols
X_train=X_train_full[my_cols]
X_valid=X_valid_full[my_cols]
X_test=X_test_full[my_cols]

In [14]:
# импорт библиотек
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [15]:
# предварительная обработка числовых данных
num_transformer=SimpleImputer(strategy='median')

In [16]:
# предварительная обработка категориальных данных
cat_transformer=Pipeline(steps=[('imputer', SimpleImputer(strategy='constant')),('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [17]:
# пакетная предварительная обработка числовых и категориальных данных
processor=ColumnTransformer(transformers=[('num', num_transformer, num_cols),('cat', cat_transformer, cat_cols)])

In [18]:
# определение модели
model = RandomForestRegressor(n_estimators=41, random_state=0)

In [19]:
# объединение кода предварительной обработки и моделирования в pipeline
my_pipeline=Pipeline(steps=[('processor',processor),('model',model)])

In [20]:
# обучение модели
my_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('processor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='median',
                                                                verbose=0),
                                                  ['PassengerId', 'Pclass',
                                                   'Age', 'SibSp', 'Parch',
                                                   'Fare']),
          

In [21]:
# получение прогноза
preds=my_pipeline.predict(X_valid)

In [22]:
preds

array([0.51219512, 0.        , 0.34146341, 0.70731707, 0.2195122 ,
       0.07317073, 0.85365854, 0.7804878 , 0.51219512, 0.6097561 ,
       0.12195122, 0.51219512, 0.02439024, 1.        , 1.        ,
       0.90243902, 0.02439024, 0.09756098, 0.02439024, 0.90243902,
       0.17073171, 0.63414634, 0.        , 0.14634146, 0.51219512,
       1.        , 0.31707317, 0.75609756, 0.95121951, 0.34146341,
       0.2195122 , 0.82926829, 0.14634146, 0.2195122 , 0.02439024,
       0.51219512, 0.09756098, 0.        , 0.09756098, 0.19512195,
       0.19512195, 0.02439024, 0.09756098, 0.09756098, 1.        ,
       0.        , 0.        , 0.92682927, 0.        , 0.12195122,
       0.09756098, 0.04878049, 1.        , 0.09756098, 0.17073171,
       0.36585366, 0.29268293, 0.        , 0.19512195, 0.        ,
       0.2195122 , 0.73170732, 0.97560976, 0.07317073, 0.7804878 ,
       0.48780488, 0.95121951, 0.09756098, 0.41463415, 0.95121951,
       1.        , 0.36585366, 0.36585366, 0.09756098, 0.12195

In [23]:
print('mean_absolute_error:', round(mean_absolute_error(y_valid, preds)*100,2))

mean_absolute_error: 24.58


Т.е. наша модель дает прогноз с ошибкой в 24,58%

In [24]:
# создание прогноза для основного файла test_data (X_test_full)
preds_test=my_pipeline.predict(X_test)